This notebook imports the preprocessed training data, as well as the test data.
It preprocesses the test data and then trains a random forest regressor.

The same random forest regressor is used to make predicitons for the test set.

Like the 'Preprocessing Training Data.ipynb', this notebook removes timestamp alltogether.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

import pandas as pd
import numpy as np

In [3]:
test = pd.read_csv("C:/Users/hclsa/Desktop/Fall2019/CMPE188/TeamProject/raw_data/test.csv")
train_preprocessed = pd.read_csv("C:/Users/hclsa/Desktop/Fall2019/CMPE188/TeamProject/train_preprocessed.csv")
train_target = pd.read_csv("C:/Users/hclsa/Desktop/Fall2019/CMPE188/TeamProject/train_target.csv")

In [4]:
building_metadata_preprocessed = pd.read_csv("C:/Users/hclsa/Desktop/Fall2019/CMPE188/TeamProject/processed_data/building_metadata_preprocessed.csv")
weather_test_preprocessed = pd.read_csv("C:/Users/hclsa/Desktop/Fall2019/CMPE188/TeamProject/processed_data/weather_test_preprocessed.csv")

In [5]:
testRow = test['row_id'].copy()
testValues = test.drop(['row_id'], axis=1)
testValues.head()

,building_id,meter,timestamp
0,0,0,2017-01-01 00:00:00
1,1,0,2017-01-01 00:00:00
2,2,0,2017-01-01 00:00:00
3,3,0,2017-01-01 00:00:00
4,4,0,2017-01-01 00:00:00


In [6]:
test_and_building = testValues.merge(building_metadata_preprocessed, on=['building_id'], how='left')

In [7]:
test_building_weather = test_and_building.merge(weather_test_preprocessed, on=['site_id', 'timestamp'], how='left')

In [8]:
test_building_weather_reduced = test_building_weather.drop(['timestamp', 'building_id'], axis=1)

In [9]:
meanImputer = SimpleImputer(missing_values=np.nan, strategy='mean')
test_final = meanImputer.fit_transform(test_building_weather_reduced)
test_final = pd.DataFrame(test_final, columns = test_building_weather_reduced.columns)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(train_preprocessed, train_target, random_state=0)

In [11]:
RFregr = RandomForestRegressor(random_state=0, n_estimators=3)
RFregr.fit(X_train, y_train)

print("Accuracy on training set: {:.4f}".format(RFregr.score(X_train, y_train)))
print("Accuracy on test set: {:.4f}".format(RFregr.score(X_test, y_test)))

C:\Users\hclsa\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Accuracy on training set: 0.8880
Accuracy on test set: 0.5858


In [12]:
PredictionRF = RFregr.predict(test_final)
PredictionRF = pd.DataFrame(PredictionRF, columns = ["meter_reading"])

PredictionRFCombined = pd.concat([testRow,PredictionRF],axis=1)

PredictionRFCombined.to_csv('PredictionRandomForestRegressor1.csv', index = False)
PredictionRFCombined.head()

,row_id,meter_reading
0,0,64.843333
1,1,20.112767
2,2,0.000000
3,3,0.000000
4,4,467.100000


In [ ]:
# 0.5858